# Support Vector Machine

In [1]:
import numpy as np
import pandas as pd
#import sklearn as sm
from sklearn import svm
import math

In [75]:
PROFILE = pd.read_csv('/home/dside/Documents/GitHub/Analysis/PROFILE.csv') # (1109, 5)
AVG_PROFILE = pd.read_csv('/home/dside/Documents/GitHub/Analysis/avg_PROFILE.csv') #(278, 4)

SVM_PRED = pd.read_csv('/home/dside/Documents/GitHub/Analysis/svm_predictors.csv') # (713, 16)
SVM_PRED = SVM_PRED.rename(columns = {'Unnamed: 0': 'date'})
AVG_SVM_PRED = pd.read_csv('/home/dside/Documents/GitHub/Analysis/avg_svm_predictors.csv') #(275, 15)

In [76]:
AVG = AVG_SVM_PRED.merge(AVG_PROFILE, on = ['mun_code'])
del AVG['Efficient_measure']
del AVG['wellfare_measure']
del AVG['mun_code']
AVG.drop_duplicates()
#AVG #(275, 15)
AVG = AVG.dropna()
AVG = AVG.reset_index() #(254, 15)
del AVG['index']
AVG

,repairs_PPE,currRatio,cash_costCoverage,OppSurplusMargin,debt_Rev,Tertiary_educ,birth_prop,EmpAdult_Prop,Citizenship_Prop,EducAttend_Prop,Female_Prop,Overcrowded_Prop,IncPoverty_Prop,Living_with_Parents_Prop,avg_profile
0,0.023720,2.482919,0.471867,0.100104,0.101412,0.069307,0.249921,0.586196,0.985233,0.572181,0.496425,0.198961,0.556065,0.215037,1
1,0.000000,1.266176,0.123189,0.073212,0.028515,0.089004,0.259899,0.785328,0.954220,0.460475,0.498950,0.135318,0.373324,0.353355,1
2,0.024592,4.183302,0.079951,0.096625,0.092087,0.030005,0.334665,0.858646,0.986474,0.330713,0.504379,0.212251,0.390721,0.419944,1
3,0.002509,6.993149,1.954877,-0.029235,0.002078,0.042270,0.274362,0.676680,0.994136,0.397358,0.488284,0.156325,0.594569,0.259822,1
4,0.004005,2.243251,0.193881,0.176914,0.026134,0.027881,0.306910,0.290302,0.997927,0.623664,0.489573,0.408396,0.794222,0.173228,2
5,0.000204,2.783084,0.092339,0.318869,0.110554,0.029580,0.284007,0.343777,0.996159,0.602654,0.488496,0.335402,0.762662,0.182638,2
6,0.000000,0.821348,0.045276,0.061398,0.123136,0.020929,0.324656,0.376967,0.977005,0.612017,0.503987,0.264157,0.763277,0.151652,2
7,0.026370,1.123713,-0.009961,0.220736,0.127911,0.026980,0.354716,0.266105,0.997467,0.655232,0.500734,0.316960,0.826224,0.167972,2
8,0.066540,0.402117,-0.064574,-0.319792,0.118795,0.018421,0.349895,0.540473,0.995190,0.472394,0.498119,0.129445,0.657563,0.222880,1
9,0.006277,3.927692,0.300726,0.031443,0.073173,0.045333,0.312677,0.570921,0.984815,0.495810,0.481671,0.113178,0.610063,0.256959,1


In [74]:
AVG.to_csv('/home/dside/Documents/GitHub/Analysis/AVG_svm.csv')

## Data Linearly Separable. 

## Argument Value Selection 

In [5]:
#Argument ranges we wish to test
error_penalty = np.arange(0.00001, 2.1, 0.1)

#Tolerence of stopping criteria:
tol = np.arange(0.00001, 0.101, 0.01)
shrinkage = ['TRUE', 'FALSE']


In [5]:
# Testing the errCost variable:

def errorPenaltyTest(TRAIN_PRED,TRAIN_RESP, TEST_PRED):
    
    curr_predictions = pd.DataFrame()
    for i in range(0, len(error_penalty)):

        currCost = error_penalty[i]

        #alter model def
        model = svm.SVC(C = currCost, kernel= 'linear', 
                       shrinking = False, probability = False, tol = 0.001, cache_size = 200, 
                       class_weight = None, verbose = False, max_iter = -1, decision_function_shape = 'ovr', 
                       random_state = None)
        #fit model
        svm_fit = model.fit(TRAIN_PRED, TRAIN_RESP)  
        
        #prediction
        prediction = model.predict(TEST_PRED)
        prediction = pd.Series(prediction)
        prediction = prediction.to_frame()
        
        curr_predictions = pd.concat([curr_predictions, prediction], axis = 1) #axis indicates by column
         
    return curr_predictions # each sublist corresponds to a different cost on error.


In [24]:
# insert function to test tolerance here
def toleranceTest(TRAIN_PRED,TRAIN_RESP, TEST_PRED):
    
    curr_predictions = pd.DataFrame()
    for i in range(0, len(tol)):

        currTol = tol[i]

        #alter model def
        model = svm.SVC(C = 0.10001, kernel= 'linear', 
                       shrinking = False, probability = False, tol = currTol, cache_size = 200, 
                       class_weight = None, verbose = False, max_iter = -1, decision_function_shape = 'ovr', 
                       random_state = None)
        #fit model
        svm_fit = model.fit(TRAIN_PRED, TRAIN_RESP)  
        
        #prediction
        prediction = model.predict(TEST_PRED)
        prediction = pd.Series(prediction)
        prediction = prediction.to_frame()
        
        curr_predictions = pd.concat([curr_predictions, prediction], axis = 1) #axis indicates by column
         
    return curr_predictions # each sublist corresponds to a different cost on error.



In [6]:
# Function to test effect of shrinkage

def shrinkage_FALSE_Test(TRAIN_PRED,TRAIN_RESP, TEST_PRED):
    curr_predictions = pd.DataFrame()
    #for i in range(0, len(shrinkage)):

        #currShrinkage = shrinkage[i]

        #alter model def
    model = svm.SVC(C = 0.10001, kernel= 'linear', 
                       shrinking = False, probability = False, tol = 0.00001 , cache_size = 200, 
                       class_weight = None, verbose = False, max_iter = -1, decision_function_shape = 'ovr', 
                       random_state = None)
        #fit model
    svm_fit = model.fit(TRAIN_PRED, TRAIN_RESP)  
        
        #prediction
    prediction = model.predict(TEST_PRED)
    prediction = pd.Series(prediction)
    prediction = prediction.to_frame()
        
    curr_predictions = pd.concat([curr_predictions, prediction], axis = 1) #axis indicates by column
         
    return curr_predictions # each sublist corresponds to a different cost on error.
    

In [69]:
# Implementing the above function within crossvalidation..
def crossValidation(data):    
    
    N = len(data)
    steps = int(N / 5)
    
    predictions_large = pd.DataFrame()

    for i in range(0,5):

        curr_test = range(i*steps, i*steps + steps)

        train = data.drop(data.index[curr_test])
        train_resp = train['avg_profile']
        train_pred =  train
        del train_pred['avg_profile']

        test =  data.iloc[curr_test]
        test_resp = test['avg_profile']
        test_pred = test
        del test_pred['avg_profile']
        
        predictions = []
        
        currFit = variableSig(TRAIN_PRED = train_pred, TRAIN_RESP = train_resp, TEST_PRED = test_pred)
        predictions_large = predictions_large.append(currFit)
    
    return predictions_large


In [9]:
AVG.shape

(254, 15)

In [28]:
AVG = AVG.iloc[0:250,0:15]
AVG.shape

(250, 15)

In [17]:
N = len(AVG)
N / 5

50.0

In [19]:
#AVG = AVG.iloc[0:250,0:15]
predictions_errorPenalty = crossValidation(AVG)
predictions_errorPenalty = predictions_errorPenalty.reset_index()
del predictions_errorPenalty['index']
predictions_errorPenalty #(250, 21)

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
4,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
7,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
8,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [25]:
predictions_tolTest = crossValidation(AVG)
predictions_tolTest = predictions_tolTest.reset_index()
del predictions_tolTest['index']
predictions_tolTest#(250, 21)

,0,0,0,0,0,0,0,0,0,0,0
0,2,2,2,2,2,2,2,2,2,2,2
1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1
3,2,2,2,2,2,2,2,2,2,2,2
4,2,2,2,2,2,2,2,2,2,2,2
5,2,2,2,2,2,2,2,2,2,2,2
6,2,2,2,2,2,2,2,2,2,2,2
7,2,2,2,2,2,2,2,2,2,2,2
8,1,1,1,1,1,1,1,1,1,1,1
9,2,2,2,2,2,2,2,2,2,2,2


In [55]:
predictions_shrinkageTRUE = crossValidation(AVG)

In [59]:
predictions_shrinkageFALSE = crossValidation(AVG)

In [26]:
# Now test the accuracy of your predictions...
Accuracy = pd.DataFrame()
zero = [{'acc':0}]
one = [{'acc':1}]
for i in range(0, len(AVG)):
    curr_Row = pd.DataFrame()
    for j in range(0,predictions_errorPenalty.shape[1]):
        if AVG['avg_profile'].iloc[i] != predictions_errorPenalty.iloc[i,j]:
            zero = pd.DataFrame(zero)  
            curr_Row = pd.concat([curr_Row,zero], axis = 1)
        if AVG['avg_profile'].iloc[i] == predictions_errorPenalty.iloc[i,j]:
            one = pd.DataFrame(one)
            curr_Row = pd.concat([curr_Row,one], axis = 1)
        
    Accuracy = Accuracy.append(curr_Row)  

# NOTEsince you have some observations that are excluded in the cross-validation, some of the mun are left out..

In [27]:
tolAccuracy = pd.DataFrame()
zero = [{'acc':0}]
one = [{'acc':1}]
for i in range(0, len(AVG)):
    curr_Row = pd.DataFrame()
    for j in range(0,predictions_tolTest.shape[1]):
        if AVG['avg_profile'].iloc[i] != predictions_tolTest.iloc[i,j]:
            zero = pd.DataFrame(zero)  
            curr_Row = pd.concat([curr_Row,zero], axis = 1)
        if AVG['avg_profile'].iloc[i] == predictions_tolTest.iloc[i,j]:
            one = pd.DataFrame(one)
            curr_Row = pd.concat([curr_Row,one], axis = 1)
        
    tolAccuracy = tolAccuracy.append(curr_Row)  


In [62]:
ShrinkTRUE_Accuracy = pd.DataFrame()
zero = [{'acc':0}]
one = [{'acc':1}]
for i in range(0, len(AVG)):
    curr_Row = pd.DataFrame()
    for j in range(0,predictions_shrinkageTRUE.shape[1]):
        if AVG['avg_profile'].iloc[i] != predictions_shrinkageTRUE.iloc[i,j]:
            zero = pd.DataFrame(zero)  
            curr_Row = pd.concat([curr_Row,zero], axis = 1)
        if AVG['avg_profile'].iloc[i] == predictions_shrinkageTRUE.iloc[i,j]:
            one = pd.DataFrame(one)
            curr_Row = pd.concat([curr_Row,one], axis = 1)
        
    ShrinkTRUE_Accuracy =ShrinkTRUE_Accuracy.append(curr_Row)  

In [63]:
ShrinkFALSE_Accuracy = pd.DataFrame()
zero = [{'acc':0}]
one = [{'acc':1}]
for i in range(0, len(AVG)):
    curr_Row = pd.DataFrame()
    for j in range(0,predictions_shrinkageFALSE.shape[1]):
        if AVG['avg_profile'].iloc[i] != predictions_shrinkageFALSE.iloc[i,j]:
            zero = pd.DataFrame(zero)  
            curr_Row = pd.concat([curr_Row,zero], axis = 1)
        if AVG['avg_profile'].iloc[i] == predictions_shrinkageFALSE.iloc[i,j]:
            one = pd.DataFrame(one)
            curr_Row = pd.concat([curr_Row,one], axis = 1)
        
    ShrinkFALSE_Accuracy =ShrinkFALSE_Accuracy.append(curr_Row)  

In [29]:
Accuracy = Accuracy.reset_index()
del Accuracy['index']
#del Accuracy['level_0']
for i in range(0,Accuracy.shape[1]):
    Accuracy.columns.values[i] = i


In [30]:
tolAccuracy = tolAccuracy.reset_index()
del tolAccuracy['index']
#del Accuracy['level_0']
for i in range(0,tolAccuracy.shape[1]):
    tolAccuracy.columns.values[i] = i


In [64]:
ShrinkTRUE_Accuracy = ShrinkTRUE_Accuracy.reset_index()
del ShrinkTRUE_Accuracy['index']
#del Accuracy['level_0']
for i in range(0,ShrinkTRUE_Accuracy.shape[1]):
    ShrinkTRUE_Accuracy.columns.values[i] = i

In [65]:
ShrinkFALSE_Accuracy = ShrinkFALSE_Accuracy.reset_index()
del ShrinkFALSE_Accuracy['index']
#del Accuracy['level_0']
for i in range(0,ShrinkFALSE_Accuracy.shape[1]):
    ShrinkFALSE_Accuracy.columns.values[i] = i

In [31]:
tolAccuracy

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1
9,0,0,0,0,0,0,0,0,0,0,0


In [32]:
Acc_Measure = Accuracy.sum(axis = 0)
#Acc_Measure = pd.Series(Acc_Measure)
Acc_Measure = Acc_Measure / 250
Acc_Measure

0     0.440
1     0.864
2     0.864
3     0.860
4     0.848
5     0.852
6     0.844
7     0.844
8     0.848
9     0.844
10    0.848
11    0.840
12    0.840
13    0.848
14    0.844
15    0.844
16    0.844
17    0.844
18    0.844
19    0.844
20    0.848
dtype: float64

In [33]:
tolAcc_Measure = tolAccuracy.sum(axis = 0)
tolAcc_Measure = tolAcc_Measure / 250
tolAcc_Measure

0     0.864
1     0.860
2     0.856
3     0.860
4     0.864
5     0.864
6     0.864
7     0.864
8     0.864
9     0.864
10    0.860
dtype: float64

In [66]:
shrinkTRUE_Measure = ShrinkTRUE_Accuracy.sum(axis = 0)
shrinkTRUE_Measure = shrinkTRUE_Measure / 250
shrinkTRUE_Measure

0    0.864
dtype: float64

In [67]:
shrinkFALSE_Measure = ShrinkFALSE_Accuracy.sum(axis = 0)
shrinkFALSE_Measure = shrinkFALSE_Measure / 250
shrinkFALSE_Measure

0    0.864
dtype: float64

In [34]:
Optimal_index = Acc_Measure.idxmax()
Optimal_ErrorPenalty = error_penalty[Optimal_index] 
Optimal_ErrorPenalty

0.10001

In [35]:
Optimal_ind = tolAcc_Measure.idxmax()
Optimal_tolerance = tol[Optimal_ind] 
Optimal_tolerance

1.0000000000000001e-05

# Evaluating Variable Importance

In [36]:
AVG_RESP = AVG['avg_profile']
AVG_PRED = AVG.drop('avg_profile', axis=1)
model = svm.SVC(C = 0.10001, kernel= 'linear', 
                       shrinking = False, probability = False, tol = 0.00001, cache_size = 200, 
                       class_weight = None, verbose = False, max_iter = -1, decision_function_shape = 'ovr', 
                       random_state = None)
#fit model
svm_fit = model.fit(AVG_PRED, AVG_RESP)


In [37]:
#svm_fit.coef_
svm_fit.support_vectors_

array([[  2.37196069e-02,   2.48291930e+00,   4.71867077e-01, ...,
          1.98961071e-01,   5.56065057e-01,   2.15037078e-01],
       [  2.50875917e-03,   6.99314942e+00,   1.95487682e+00, ...,
          1.56325398e-01,   5.94568573e-01,   2.59822030e-01],
       [  6.65403743e-02,   4.02117201e-01,  -6.45738261e-02, ...,
          1.29444851e-01,   6.57563429e-01,   2.22879966e-01],
       ..., 
       [  1.69796480e-02,   1.06609011e+00,  -3.00535971e-02, ...,
          1.80727117e-01,   7.27272727e-01,   1.85235378e-01],
       [  0.00000000e+00,   1.17640574e+00,   1.90845500e-02, ...,
          2.00120291e-01,   8.03865007e-01,   1.29737609e-01],
       [  0.00000000e+00,   7.16495661e-01,   4.48232458e-02, ...,
          1.87738279e-01,   7.33586961e-01,   2.02840657e-01]])

In [38]:
svm_fit.coef_

array([[ 0.00545528,  0.07827291, -0.05971987,  1.0790047 , -0.26671842,
        -0.08324468,  0.01343069, -1.79055742,  0.04726301,  1.09911835,
         0.01487754,  0.41371896,  1.25197891, -0.62712093]])

In [39]:
dir(svm_fit) 

['C',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_check_proba',
 '_compute_kernel',
 '_decision_function',
 '_dense_decision_function',
 '_dense_fit',
 '_dense_predict',
 '_dense_predict_proba',
 '_dual_coef_',
 '_estimator_type',
 '_gamma',
 '_get_coef',
 '_get_param_names',
 '_impl',
 '_intercept_',
 '_pairwise',
 '_predict_log_proba',
 '_predict_proba',
 '_sparse',
 '_sparse_decision_function',
 '_sparse_fit',
 '_sparse_kernels',
 '_sparse_predict',
 '_sparse_predict_proba',
 '_validate_for_predict',
 '_validate_tar

## Backward Selection 

In [34]:
# Final Model - fit and test accuracy
model = svm.SVC(C = 0.10001, kernel= 'linear', 
                       shrinking = False, tol = 0.00001 , cache_size = 200, 
                       class_weight = None, verbose = False, max_iter = -1, decision_function_shape = 'ovr', 
                       random_state = None)

In [46]:
def variableSig(TRAIN_PRED,TRAIN_RESP, TEST_PRED):
    
    curr_predictions = pd.DataFrame()
    
    for i in range(0, TRAIN_PRED.shape[1]): # move through all columns
        
        #drop the relevant column
        Curr_train_pred = TRAIN_PRED.drop(TRAIN_PRED.columns[i], axis = 1)
        Curr_test_pred = TEST_PRED.drop(TEST_PRED.columns[i], axis = 1)
        
        #fit model
        svm_fit = model.fit(Curr_train_pred, TRAIN_RESP)  
        
        #prediction
        prediction = model.predict(Curr_test_pred)
        prediction = pd.Series(prediction)
        prediction = prediction.to_frame()
        
        curr_predictions = pd.concat([curr_predictions, prediction], axis = 1)
         
    return curr_predictions


In [19]:
# Implementing the above function within crossvalidation..
def crossValidation(data):    
    
    N = len(data)
    steps = int(N / 5) # divide data into 5 parts
    
    predictions_large = pd.DataFrame()

    for i in range(0,5):

        curr_test = range(i*steps, i*steps + steps)

        train = data.drop(data.index[curr_test])
        train_resp = train['avg_profile']
        train_pred =  train
        del train_pred['avg_profile']

        test =  data.iloc[curr_test]
        test_resp = test['avg_profile']
        test_pred = test
        del test_pred['avg_profile']
        
        #predictions = []
        
        currFit = variableSig(TRAIN_PRED = train_pred, TRAIN_RESP = train_resp, TEST_PRED = test_pred)
        predictions_large = predictions_large.append(currFit)
    
    return predictions_large


In [36]:
def Accuracy(currPred):
    Rem_Accuracy = pd.DataFrame()
    zero = [{'acc':0}]
    one = [{'acc':1}]
    for i in range(0, len(currPred)):
        curr_Row = pd.DataFrame()
        for j in range(0, currPred.shape[1]):
            if AVG['avg_profile'].iloc[i] != currPred.iloc[i,j]:
                zero = pd.DataFrame(zero)  
                curr_Row = pd.concat([curr_Row,zero], axis = 1)
            if AVG['avg_profile'].iloc[i] == currPred.iloc[i,j]:
                one = pd.DataFrame(one)
                curr_Row = pd.concat([curr_Row,one], axis = 1)

        Rem_Accuracy = Rem_Accuracy.append(curr_Row) 
    return Rem_Accuracy


In [69]:
def backwardSelection(data):
    
    var_importance = []
    currRange = AVG.shape[1] - 2
    for i in range(0, currRange): #
        
        currPred = crossValidation(data)
        #currPred = currPred.reset_index()
        currAccuracy = Accuracy(currPred)
        
        currAccuracy = currAccuracy.reset_index()
        del currAccuracy['index']

        for j in range(0,currAccuracy.shape[1]):
            currAccuracy.columns.values[j] = j
            
        RemAcc_Measure = currAccuracy.sum(axis = 0)
        RemAcc_Measure = RemAcc_Measure / len(data)
        
        RemOptimal_index = RemAcc_Measure.idxmax()
        #extract column name
        colname = data.columns[RemOptimal_index] 
        var_importance.append(colname)
        
        data = data.drop(data.columns[RemOptimal_index], axis = 1)
    
    var_importance.append(data.columns[0])
        
    return(var_importance)        

In [70]:
AVG_var_imp = backwardSelection(data = AVG)

In [71]:
AVG_var_imp

['currRatio',
 'Overcrowded_Prop',
 'cash_costCoverage',
 'repairs_PPE',
 'Tertiary_educ',
 'birth_prop',
 'Citizenship_Prop',
 'Female_Prop',
 'Living_with_Parents_Prop',
 'debt_Rev',
 'EducAttend_Prop',
 'IncPoverty_Prop',
 'OppSurplusMargin',
 'EmpAdult_Prop']

In [194]:
Removal_one = crossValidation(AVG_removal)

In [78]:
Rem_Accuracy

,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,0,1,1,1,1,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,0,1,1,1,1,0,1
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [196]:
Rem_Accuracy = Rem_Accuracy.reset_index()
del Rem_Accuracy['index']

for i in range(0,Rem_Accuracy.shape[1]):
    Rem_Accuracy.columns.values[i] = i

Rem_Accuracy

,0,1
0,1,1
1,1,1
2,1,1
3,1,1
4,1,0
5,1,0
6,1,0
7,1,0
8,1,1
9,1,1


In [197]:
RemAcc_Measure = Rem_Accuracy.sum(axis = 0)
#Acc_Measure = pd.Series(Acc_Measure)
RemAcc_Measure = RemAcc_Measure / 250
RemAcc_Measure

0    0.724
1    0.456
dtype: float64

In [198]:
RemOptimal_index = RemAcc_Measure.idxmax()
Optimal_Removal = RemAcc_Measure[RemOptimal_index] 
RemOptimal_index # removing currRatio produces the best accuracy (1, 10, 1, 0, 2, 2, 3, 4, 5, 1, 2, 2, 0)

0

In [199]:
AVG_removal = AVG.drop(AVG.columns[1], axis=1) #currRatio 
AVG_removal = AVG_removal.drop(AVG_removal.columns[10], axis = 1) #Overcrowded_Prop 
AVG_removal = AVG_removal.drop(AVG_removal.columns[1], axis = 1) #cash_costCoverage
AVG_removal = AVG_removal.drop(AVG_removal.columns[0], axis = 1) #repairs_PPE
AVG_removal = AVG_removal.drop(AVG_removal.columns[2], axis = 1) #Tertiary_educ
AVG_removal = AVG_removal.drop(AVG_removal.columns[2], axis = 1) #birth_prop
AVG_removal = AVG_removal.drop(AVG_removal.columns[3], axis = 1) #Citizenship_Prop
AVG_removal = AVG_removal.drop(AVG_removal.columns[4], axis = 1) #Female_Prop
AVG_removal = AVG_removal.drop(AVG_removal.columns[5], axis = 1) #Living_with_Parents_Prop
AVG_removal = AVG_removal.drop(AVG_removal.columns[1], axis = 1) #debt_Rev
AVG_removal = AVG_removal.drop(AVG_removal.columns[2], axis = 1) #EducAttend_Prop
AVG_removal = AVG_removal.drop(AVG_removal.columns[2], axis = 1) #IncPoverty_Prop
AVG_removal = AVG_removal.drop(AVG_removal.columns[0], axis = 1) #OppSurplusMargin
#EmpAdult_Prop

In [200]:
AVG_removal

,EmpAdult_Prop,avg_profile
0,0.586196,1
1,0.785328,1
2,0.858646,1
3,0.676680,1
4,0.290302,2
5,0.343777,2
6,0.376967,2
7,0.266105,2
8,0.540473,1
9,0.570921,1


In [ ]:

#np.any(np.isnan(AVG))
#np.isfinite(AVG) 
#AVG.dtypes
#np.any(np.isnan(predictions_errorPenalty))

In [77]:
AVG_resp = AVG['avg_profile']
AVG_pred = AVG.drop(AVG['avg_profile'])

model = svm.SVC(C = 0.10001, kernel= 'linear', 
                       shrinking = False, tol = 0.00001 , cache_size = 200, 
                       class_weight = None, verbose = False, max_iter = -1, decision_function_shape = 'ovr', 
                       random_state = None)


In [78]:
svm_fit = model.fit(AVG_pred, AVG_resp)

ValueError: Found input variables with inconsistent numbers of samples: [252, 254]